In [ ]:
import numpy as np
import mne
import scipy.io as sio
import hdf5storage
import os

In [ ]:
#Test with a single night

path_raw_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple/S_01/night_1/EEG_raw.mat'
path_cleaned_data = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned/S_01/night_1/EEG_raw.mat'

print('loading data')
data_raw = hdf5storage.loadmat(path_raw_data)
print("raw data loaded")

#data_cleaned = hdf5storage.loadmat(path_cleaned_data)['data']
#print("clean data loaded")

In [ ]:
print('debug')

In [ ]:

labels = data_raw['chanlabels'].reshape(25).tolist() #channel labels needed to load data with MNE

#A line to first load a nights data into mne, then resample to 250 hz, then get the numpy array out again with ._data
data = mne.io.RawArray(data_raw['data'], mne.create_info(labels,500)).resample(250)._data


In [ ]:
clean_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple_cleaned'
raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'

#Run through all the cleaned EEG files
for subdir, dirs, files in os.walk(raw_data_dir):
    for file in files:
        if "EEG" in file:
            print(os.path.join(subdir, file))

            np.save(f'{os.path.join(subdir)}/artefact_annotations', annotated_data)
            np.save(f'{os.path.join(raw_dir)}/artefact_annotations', annotated_data)

            #save the shape of the array as well
            np.save(f'{os.path.join(subdir)}/data_shape', annotated_data.shape)
            np.save(f'{os.path.join(raw_dir)}/data_shape', annotated_data.shape)


            print(f'Done: {subdir}')